In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwings as xw
import string

In [2]:
# Dictionary to locate the parameters inside the spreadsheet.

dictpos = {"Gamma":[6,4],"lambda":[7,4],"Mtc":[14,4],"N":[15,4],
           "Xtc": [16,4],"H0":[17,4],"Hy":[18,4], "Gmax_p0":[21,4],
           "G_exp": [22,4], "nu":[23,4],"Psi_0":[27,4],"p0":[29,4],
           "K0": [30,4], "OCR": [32,4]}

In [3]:
# Function to change the parameters and run the macros. The inputs are:
# a) final_comp: input parameters as a numpy array of shape (1,14). The parameters need to be inserted
#    in the same order as dictpos.keys(), i.e., ['Gamma', 'lambda', 'Mtc', 'N', 'Xtc', 'H0', 'Hy', 
#    'Gmax_p0', 'G_exp', 'nu', 'Psi_0', 'p0', 'K0', 'OCR'].
# b) dictpos: dictionary to locate the parameters inside the spreadsheet
# c) path_root: path of the spreadsheet "NorTxl.xlsm", obtained at http://www.crcpress.com/product/isbn/9781482213683
# d) type_v: type of the simulation (either "Drained" or "Undrained")
# The function outputs two entities:
# - dictionary containing the parameters inserted to run the simulation;
# - pandas dataframe with simulation results (which are located inside the "Txl SimResults" tab). The columns are:
#   [ep1, epV, p, q, e, pi/p, (pi/p)max, y, Dp, eta].

def run_NorSand(final_comp,dictpos,path_root,type_v):
    letters = list(string.ascii_uppercase)
    wb = xw.Book(path_root)  
    app = wb.app
    macro_vba = app.macro("'NorTxl.xlsm'!RunSim")
    macro_vba_type = app.macro("'NorTxl.xlsm'!ChangeSimMode")
    ws = wb.sheets["Params & Plots"]
    results_comp = []
    for new_v in final_comp:
        for nv,ps in zip(new_v,dictpos.values()):
            pl,pc = ps
            pfinal = letters[pc-1]+str(pl)
            ws[pfinal].value = nv
        if ws["D34"].value == type_v:
            pass
        else:
            macro_vba_type()
        macro_vba()
        ws_results = wb.sheets["Txl SimResults"]
        np_arr = (ws_results['A4'].expand('table')).value                                 
        dd = np.array(np_arr).astype(np.float64)
        dict_inpts = {}
        for keyv,pvalu in zip(dictpos.keys(),new_v.astype(np.float64)):
            dict_inpts[keyv] = pvalu
        dict_inpts["Type"] = type_v
        return dict_inpts,pd.DataFrame(dd)

In [10]:
# Example:
path_root = "D:\\NorSandTXL_H5\\NorTxl.xlsm"
type_v = "Drained"
final_comp = np.array([[1.1,0.04,1.3,0.4,3,200,350,50,0.3,0.2,0.1,500,0.95,1.2]])
a1,a2 = run_NorSand(final_comp,dictpos,path_root,type_v)

In [11]:
a1

{'Gamma': 1.1,
 'lambda': 0.04,
 'Mtc': 1.3,
 'N': 0.4,
 'Xtc': 3.0,
 'H0': 200.0,
 'Hy': 350.0,
 'Gmax_p0': 50.0,
 'G_exp': 0.3,
 'nu': 0.2,
 'Psi_0': 0.1,
 'p0': 500.0,
 'K0': 0.95,
 'OCR': 1.2,
 'Type': 'Drained'}

In [12]:
a2

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.000000,500.000000,25.862069,0.951416,0.459374,0.793923,0.100000,0.770230,0.051724
1,0.072409,0.044378,529.847258,113.052071,0.950550,0.433497,0.793923,0.100000,0.770230,0.213367
2,0.085730,0.053719,532.500333,121.011294,0.950367,0.438151,0.791265,0.101453,1.086633,0.227251
3,0.098848,0.062883,535.084441,128.763620,0.950189,0.442688,0.791233,0.101471,1.072749,0.240642
4,0.111772,0.071877,537.603105,136.319610,0.950013,0.447112,0.791206,0.101486,1.059358,0.253569
...,...,...,...,...,...,...,...,...,...,...
3995,19.825467,4.490866,819.705758,982.627570,0.863780,0.925076,0.928499,0.032147,0.101272,1.198756
3996,19.829635,4.491366,819.718809,982.666722,0.863770,0.925096,0.928519,0.032138,0.101244,1.198785
3997,19.833802,4.491865,819.731856,982.705863,0.863761,0.925117,0.928538,0.032129,0.101215,1.198814
3998,19.837969,4.492365,819.744900,982.744994,0.863751,0.925137,0.928558,0.032120,0.101186,1.198842
